In [1]:
import numpy as np
from PIL import Image, ImageDraw
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression

In [2]:
# Open the image
png = Image.open("./red.png")
image = np.array(png)

In [3]:
# Hyperparameters
red_score = 2.25
green_score = -3.
blue_score = -3.

orange_threshold = 185

In [4]:
# Peform linear transformation on the image to assign an orange score to each pixel
size = (image.shape[0], image.shape[1])
r = np.ones(size) * red_score
g = np.ones(size) * green_score
b = np.ones(size) * blue_score

transformation = np.dstack((r,g,b))
transformation = np.multiply(image, transformation)

orange_scores = np.sum(transformation, axis = -1)

In [5]:
# Use the orange scores and treshold to determine which pixels are orange
orange_image = (orange_scores > orange_threshold) * 1

# Get the indicies of all of the orange pixels
orange_pixels = np.nonzero(orange_image)
orange_pixels = np.transpose(orange_pixels)

In [6]:
# Use Gaussian Mixture algorithm to quickly classify orange pixels into two lines
gm = GaussianMixture(n_components=2).fit(orange_pixels)
labels = gm.predict(orange_pixels)

In [7]:
# Seperate each line of pixels into two lists
class1 = np.nonzero(labels)
class1 = orange_pixels[class1]

labels = np.abs(labels - 1)

class2 = np.nonzero(labels)
class2 = orange_pixels[class2]

In [8]:
# Get equation of both lines using linear regression
x = class1[:,0].reshape(-1,1)
y = class1[:,1].reshape(-1,1)
line1 = LinearRegression().fit(x,y)
x = class2[:,0].reshape(-1,1)
y = class2[:,1].reshape(-1,1)
line2 = LinearRegression().fit(x,y)

In [9]:
print(f'First line: y = {line1.coef_[0,0]}x + {line1.intercept_[0]}')
print(f'Second line: y = {line2.coef_[0,0]}x + {line2.intercept_[0]}')

First line: y = 0.45953360631204626x + 828.577653414413
Second line: y = -0.4639343776083791x + 1025.9228626319796


In [10]:
# Calculate where to start and finish drawing the the lines
image_height = png.size[1]

xIntercept1 = -line1.intercept_[0] / line1.coef_[0,0]
xIntercept2 = -line2.intercept_[0] / line2.coef_[0,0]
topIntercept1 = (image_height - line1.intercept_[0]) / line1.coef_[0,0]
topIntercept2 = (image_height - line2.intercept_[0]) / line2.coef_[0,0]

# Create the lines
toDraw1 = ((0, xIntercept1), (image_height, topIntercept1))
toDraw2 = ((0, xIntercept2), (image_height, topIntercept2))

In [11]:
# Draw the lines to the png and save the results
draw = ImageDraw.Draw(png)
draw.line(toDraw1, fill='red', width=10)
draw.line(toDraw2, fill='red', width=10)
png.save('./answer.png')